# Задание к занятию «Рекомендации на основе содержания»

Описание задания:

Что делать

Построить рекомендации (регрессия, предсказываем оценку) на фичах:

   - TF-IDF на тегах и жанрах
   - средние оценки пользователя и фильма

Оценить RMSE на тестовой выборка

 

Какие данные

Датасет MovieLens (ml_latest)

## Установка пакетов и загрузка данных

In [0]:
# когда хочется обнулить текущее состояние контейнера
#!kill -9 -1

In [2]:
!cat /etc/lsb-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=17.10
DISTRIB_CODENAME=artful
DISTRIB_DESCRIPTION="Ubuntu 17.10"


In [3]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest.zip

--2018-06-02 22:28:29--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248434223 (237M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 236.92M  69.3MB/s    in 3.7s    

2018-06-02 22:28:33 (64.8 MB/s) - ‘ml-latest.zip’ saved [248434223/248434223]



In [4]:
!ls

datalab  ml-latest.zip


In [5]:
!unzip ml-latest.zip

Archive:  ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/links.csv     
  inflating: ml-latest/movies.csv    
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/tags.csv      


In [6]:
DATA_DIR = "ml-latest"
!ls {DATA_DIR}

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


In [7]:
!pip install pyspark

    100% |████████████████████████████████| 211.9MB 78kB/s 
    100% |████████████████████████████████| 194kB 19.0MB/s 
  Running setup.py bdist_wheel for pyspark ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

 - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/d9/db/ff/e6f3a8a564163ea64bc2072357e77b3404d10f91be48352796
Successfully built pyspark


In [8]:
!free -g

              total        used        free      shared  buff/cache   available
Mem:             12           0           4           0           8          12
Swap:             0           0           0


In [9]:
!apt-get install software-properties-common python-software-properties
!add-apt-repository -y ppa:webupd8team/java
!echo debconf shared/accepted-oracle-license-v1-1 select true | debconf-set-selections
!echo debconf shared/accepted-oracle-license-v1-1 seen true | debconf-set-selections
!apt-get update
!apt-get -y install oracle-java8-installer

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cron dbus dirmngr distro-info-data gir1.2-glib-2.0 iso-codes libapparmor1
  libdbus-1-3 libdbus-glib-1-2 libgirepository-1.0-1 lsb-release python-apt
  python-apt-common python-pycurl python3-apt python3-dbus python3-gi
  python3-software-properties unattended-upgrades
Suggested packages:
  anacron logrotate checksecurity exim4 | postfix | mail-transport-agent
  default-dbus-session-bus | dbus-session-bus tor isoquery lsb python-apt-dbg
  python-apt-doc libcurl4-gnutls-dev python-pycurl-dbg python-pycurl-doc
  python3-apt-dbg python-dbus-doc python3-dbus-dbg bsd-mailx
  mail-transport-agent needrestart
The following NEW packages will be installed:
  cron dbus dirmngr distro-info-data gir1.2-glib-2.0 iso-codes libapparmor1
  libdbus-1-3 libdbus-glib-1-2 libgirepository-1.0-1 lsb-release python-apt
  python-apt-common python-pycurl python-

Selecting previously unselected package iso-codes.
Preparing to unpack .../09-iso-codes_3.75-1_all.deb ...
Unpacking iso-codes (3.75-1) ...
Selecting previously unselected package libdbus-glib-1-2:amd64.
Preparing to unpack .../10-libdbus-glib-1-2_0.108-2_amd64.deb ...
Unpacking libdbus-glib-1-2:amd64 (0.108-2) ...
Selecting previously unselected package python-apt-common.
Preparing to unpack .../11-python-apt-common_1.4.0~beta3build2_all.deb ...
Unpacking python-apt-common (1.4.0~beta3build2) ...
Selecting previously unselected package python3-apt.
Preparing to unpack .../12-python3-apt_1.4.0~beta3build2_amd64.deb ...
Unpacking python3-apt (1.4.0~beta3build2) ...
Selecting previously unselected package python3-dbus.
Preparing to unpack .../13-python3-dbus_1.2.4-1build3_amd64.deb ...
Unpacking python3-dbus (1.2.4-1build3) ...
Selecting previously unselected package python3-gi.
Preparing to unpack .../14-python3-gi_3.24.1-2build1_amd64.deb ...
Unpacking python3-gi (3.24.1-2build1) ...
S

gpg: keybox '/tmp/tmpldetf5mg/pubring.gpg' created
gpg: /tmp/tmpldetf5mg/trustdb.gpg: trustdb created
gpg: key C2518248EEA14886: public key "Launchpad VLC" imported
gpg: no ultimately trusted keys found
gpg: Total number processed: 1
gpg:               imported: 1
OK
Hit:1 http://archive.ubuntu.com/ubuntu artful InRelease
Get:2 http://security.ubuntu.com/ubuntu artful-security InRelease [83.2 kB]
Get:3 http://ppa.launchpad.net/webupd8team/java/ubuntu artful InRelease [15.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu artful-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu artful-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/webupd8team/java/ubuntu artful/main amd64 Packages [1,549 B]
Get:7 http://security.ubuntu.com/ubuntu artful-security/universe Sources [20.5 kB]
Get:8 http://security.ubuntu.com/ubuntu artful-security/main amd64 Packages [222 kB]
Get:9 http://archive.ubuntu.com/ubuntu artful-updates/universe Sources [43.2 kB]
Get:10 http://security.ubun

Setting up java-common (0.59ubuntu1) ...
Setting up oracle-java8-installer (8u171-1~webupd8~0) ...
No /var/cache/oracle-jdk8-installer/wgetrc file found.
Creating /var/cache/oracle-jdk8-installer/wgetrc and
using default oracle-java8-installer wgetrc settings for it.
--2018-06-02 22:30:35--  http://download.oracle.com/otn-pub/java/jdk/8u171-b11/512cd62ec5174c3487ac17c61aaa89e8/jdk-8u171-linux-x64.tar.gz
Resolving download.oracle.com (download.oracle.com)... 23.195.252.119
Connecting to download.oracle.com (download.oracle.com)|23.195.252.119|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://edelivery.oracle.com/otn-pub/java/jdk/8u171-b11/512cd62ec5174c3487ac17c61aaa89e8/jdk-8u171-linux-x64.tar.gz [following]
--2018-06-02 22:30:35--  https://edelivery.oracle.com/otn-pub/java/jdk/8u171-b11/512cd62ec5174c3487ac17c61aaa89e8/jdk-8u171-linux-x64.tar.gz
Resolving edelivery.oracle.com (edelivery.oracle.com)... 23.7.36.127, 2610:18:1300:18f::2d3e,

...... ........ ........ ........ ........ ........ 85%  135M 0s
159744K ........ ........ ........ ........ ........ ........ 87%  135M 0s
162816K ........ ........ ........ ........ ........ ........ 88%  110M 0s
165888K ........ ........ ........ ........ ........ ........ 90%  125M 0s
168960K ........ ........ ........ ........ ........ ........ 92%  134M 0s
172032K ........ ........ ........ ........ ........ ........ 93%  135M 0s
175104K ........ ........ ........ ........ ........ ........ 95%  117M 0s
178176K ........ ........ ........ ........ ........ ........ 97%  118M 0s
181248K ........ ........ ........ ........ ........ ........ 98%  119M 0s
184320K ........ ........ ........ ........                  100%  178M=1.6s

2018-06-02 22:30:37 (116 MB/s) - ‘jdk-8u171-linux-x64.tar.gz’ saved [190890122/190890122]

Download done.
Removing outdated cached downloads...
update-alternatives: error: no alternatives for java
update-alternatives: using /usr/lib/jvm/java-8-oracle/jre/bi

Selecting previously unselected package libxfont1:amd64.
Preparing to unpack .../3-libxfont1_1%3a1.5.2-4ubuntu1.1_amd64.deb ...
Unpacking libxfont1:amd64 (1:1.5.2-4ubuntu1.1) ...
Selecting previously unselected package xfonts-encodings.
Preparing to unpack .../4-xfonts-encodings_1%3a1.0.4-2_all.deb ...
Unpacking xfonts-encodings (1:1.0.4-2) ...
Selecting previously unselected package xfonts-utils.
Preparing to unpack .../5-xfonts-utils_1%3a7.7+4_amd64.deb ...
Unpacking xfonts-utils (1:7.7+4) ...
Selecting previously unselected package gsfonts-x11.
Preparing to unpack .../6-gsfonts-x11_0.24_all.deb ...
Unpacking gsfonts-x11 (0.24) ...
Processing triggers for mime-support (3.60ubuntu1) ...
Setting up gsfonts (1:8.11+urwcyr1.0.7~pre44-4.3) ...
Setting up oracle-java8-set-default (8u171-1~webupd8~0) ...
Setting up xfonts-encodings (1:1.0.4-2) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up libfontenc1:amd64 (1:1.1.3-1) ...
Setting up libxfont1:amd64 (1:1.5.2-4ubuntu1.

## Подготовка данных

### Средние оценки

In [0]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config("spark.driver.memory", "10g")
    .config("spark.ui.enabled", "false")
    .master("local[*]")
    .getOrCreate()
)
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")

In [0]:
import os
import pyspark.sql.functions as sql_func

ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "ratings.csv"),
        header=True,
        inferSchema=True
    )
    .sample(withReplacement=False, fraction=0.2, seed=0)
    .withColumn("rating_datetime", sql_func.from_unixtime("timestamp"))
    .drop("timestamp")
    .cache()
)

In [0]:
movie_ratings = (
    ratings
    .groupBy("movieId")
    .agg(sql_func.avg("rating").alias("avg_movie_rating"))
    .cache()
)

user_ratings = (
    ratings
    .groupBy("userId")
    .agg(sql_func.avg("rating").alias("avg_user_rating"))
    .cache()
)

### TF-IDF признаки

In [0]:
import os
import pyspark.sql.functions as sql_func

# будем использовать не только теги, но и информацию о жанрах
movies = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "movies.csv"),
        header=True,
        inferSchema=True
    )
    .withColumn("genres_list", sql_func.split("genres", '\|'))
    .select("movieId", "title", "genres_list")
    .cache()
)

raw_tags = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "tags.csv"),
        header=True,
        inferSchema=True
    )
    .cache()
)


In [0]:
tags = (
    raw_tags
    # теги могут различаться только регистром,
    # поэтому приведём их все к верхнему
    .select(
        sql_func.col("movieId"),
        sql_func.upper(sql_func.col("tag")).alias("tag")
    )
    .groupBy("movieId")
    .agg(sql_func.collect_list("tag").alias("tags_list"))
    .join(movies, "movieId", "right")
    .cache()
)

In [0]:
# объединим теги и жанры в единое пространство текстовых фич
from pyspark.sql.types import ArrayType, StringType

# в Spark нет некоторых полезных функций, но легко можно создавать свои
# в частности, мы хотим привести все жанры также к верхнему регистру
list_concat = sql_func.udf(
    lambda one_list, another_list:
        [str.upper(elem) for elem in one_list] + (another_list if another_list else []),
    returnType=ArrayType(StringType())
)

content_features = (
    tags
    .select(
        "movieID",
        "title",
        list_concat("genres_list", "tags_list").alias("content_features")
    )
    .cache()
)

In [16]:
content_features.show(2, truncate=False)

+-------+--------------------------------+-------------------------------------------------------------+
|movieID|title                           |content_features                                             |
+-------+--------------------------------+-------------------------------------------------------------+
|148    |Awfully Big Adventure, An (1995)|[DRAMA, CATCHY, NUDITY (TOPLESS), NUDITY (TOPLESS - NOTABLE)]|
|463    |Guilty as Sin (1993)            |[CRIME, DRAMA, THRILLER]                                     |
+-------+--------------------------------+-------------------------------------------------------------+
only showing top 2 rows



In [0]:
# посчитаем частоты встречаемости для тегов для всех фильмов
from pyspark.ml.feature import HashingTF

term_frequencies = HashingTF(
    # от каждого тега будет вычислен хэш
    # и по факту мы будем считать частоты бакетов хэшей, а не самих тегов
    numFeatures=1024,
    inputCol="content_features",
    outputCol="term_frequencies"
).transform(content_features).cache()

In [18]:
term_frequencies.show(2, truncate=False)

+-------+--------------------------------+-------------------------------------------------------------+------------------------------------------+
|movieID|title                           |content_features                                             |term_frequencies                          |
+-------+--------------------------------+-------------------------------------------------------------+------------------------------------------+
|148    |Awfully Big Adventure, An (1995)|[DRAMA, CATCHY, NUDITY (TOPLESS), NUDITY (TOPLESS - NOTABLE)]|(1024,[434,437,572,865],[1.0,1.0,1.0,1.0])|
|463    |Guilty as Sin (1993)            |[CRIME, DRAMA, THRILLER]                                     |(1024,[263,434,968],[1.0,1.0,1.0])        |
+-------+--------------------------------+-------------------------------------------------------------+------------------------------------------+
only showing top 2 rows



In [0]:
from pyspark.ml.feature import IDF

idf_model = IDF(
    inputCol="term_frequencies",
    outputCol="tf_idf",
    minDocFreq=2
).fit(term_frequencies)
tf_idf = (
    idf_model
    .transform(term_frequencies)
    .select("movieId", "title", "tf_idf")
    .cache()
)

In [20]:
tf_idf.show(2, truncate=False)

+-------+--------------------------------+-------------------------------------------------------------------------------------------------+
|movieId|title                           |tf_idf                                                                                           |
+-------+--------------------------------+-------------------------------------------------------------------------------------------------+
|148    |Awfully Big Adventure, An (1995)|(1024,[434,437,572,865],[0.8281629655111662,3.3299385165264987,6.137879757482,4.774574914586808])|
|463    |Guilty as Sin (1993)            |(1024,[263,434,968],[1.8844903067285101,0.8281629655111662,2.3258445214092895])                  |
+-------+--------------------------------+-------------------------------------------------------------------------------------------------+
only showing top 2 rows



## Линейная регрессия

### Признаки - средние оценки

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [0]:
from pyspark.sql import DataFrame
import numpy as np


def simple_evaluate(df: DataFrame) -> float:
    return np.sqrt(
        df
        .selectExpr("""
            CASE
                WHEN prediction > 5 THEN 5
                WHEN prediction < 0.5 THEN 0.5
                ELSE prediction
            END AS prediction
        """, "label")
        .select(
            sql_func.pow(sql_func.col("label") - sql_func.col("prediction"), 2)
            .alias("squared_error")
        )
        .agg(sql_func.avg("squared_error"))
        .first()[0]
    )

In [0]:
df_full = (
    VectorAssembler(
        inputCols = ["avg_movie_rating", "avg_user_rating"],
        outputCol = "features"
    ).transform(
        ratings
        .join(movie_ratings, "movieId")
        .join(user_ratings, "userId")
    )
    .withColumnRenamed("rating", "label")
    .select("movieId", "userId", "label", "features")
    .cache()
)

In [24]:
train_data, test_data = df_full.randomSplit([0.7, 0.3], seed=42)
train_data.checkpoint()
test_data.checkpoint()

DataFrame[movieId: int, userId: int, label: double, features: vector]

In [25]:
print((train_data.count(), len(train_data.columns)))
print((test_data.count(), len(test_data.columns)))

(3643107, 4)
(1559457, 4)


In [26]:
train_data.show(2, truncate=False)

+-------+------+-----+--------------------------------------+
|movieId|userId|label|features                              |
+-------+------+-----+--------------------------------------+
|1      |3918  |4.0  |[3.8875322311542546,3.803030303030303]|
|1      |34239 |3.0  |[3.8875322311542546,3.642857142857143]|
+-------+------+-----+--------------------------------------+
only showing top 2 rows



In [32]:
linear_model = LinearRegression().fit(train_data)
train_prediction = linear_model.transform(train_data)
print("ошибка предсказания на train:", simple_evaluate(train_prediction))

#train_prediction.show(2)

ошибка предсказания на train: 0.8533505869451439


In [33]:
test_prediction = linear_model.transform(test_data)

print("ошибка предсказания на test:", simple_evaluate(test_prediction))

ошибка предсказания на test: 0.854074488878677


### Признаки - средние оценки + TFIDF

In [0]:
df_full = (
    VectorAssembler(
        inputCols = ["avg_movie_rating", "avg_user_rating", "tf_idf"],
        outputCol = "features"
    ).transform(
        ratings
        .join(movie_ratings, "movieId")
        .join(user_ratings, "userId")
        .join(tf_idf, "movieId")
    )
    .withColumnRenamed("rating", "label")
    .select("movieId", "userId", "label", "features")
    .cache()
)

In [35]:
df_full.show(2, truncate=False)

+-------+------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [36]:
train_data, test_data = df_full.randomSplit([0.7, 0.3], seed=42)
train_data.checkpoint()
test_data.checkpoint()

DataFrame[movieId: int, userId: int, label: double, features: vector]

In [37]:
print((train_data.count(), len(train_data.columns)))
print((test_data.count(), len(test_data.columns)))

(3641644, 4)
(1560920, 4)


In [38]:
linear_model = LinearRegression().fit(train_data)
train_prediction = linear_model.transform(train_data)
print("ошибка предсказания на train:", simple_evaluate(train_prediction))

ошибка предсказания на train: 0.8527723848919534


In [39]:
test_prediction = linear_model.transform(test_data)

print("ошибка предсказания на test:", simple_evaluate(test_prediction))

ошибка предсказания на test: 0.8528658070858727


In [40]:
linear_model.coefficients.shape

(1026,)

## Выводы

Добавление TFIDF признаков позволило уменьшить RMSE с 

0.854074488878677

до

0.8528658070858727